In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install open_clip_torch

In [ ]:
!gdown 1d4v9MsYBdrpcOmup0CND75Y-Hexmo3-t
!unzip -qq "/content/FULL [Heineken Vietnam] Developer Resources-20240603T132806Z-001.zip" -d "/content/dataset"
!rm "/content/FULL [Heineken Vietnam] Developer Resources-20240603T132806Z-001.zip"


Downloading...
From (original): https://drive.google.com/uc?id=1d4v9MsYBdrpcOmup0CND75Y-Hexmo3-t
From (redirected): https://drive.google.com/uc?id=1d4v9MsYBdrpcOmup0CND75Y-Hexmo3-t&confirm=t&uuid=5c0e5659-fdc6-4109-9a24-236d4fcaa46e
To: /content/FULL [Heineken Vietnam] Developer Resources-20240603T132806Z-001.zip
100% 1.10G/1.10G [00:25<00:00, 43.6MB/s]


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os
import torch
import open_clip as clip
from PIL import Image
device = "cuda" if torch.cuda.is_available() else "cpu"

In [99]:
model, _, preprocess = clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k', device=device)
model.eval()
tokenizer = clip.get_tokenizer('ViT-B-32')
context = ["in the outdoor_venue",
           "People are shopping",
           "in the bar or the pub or the night_club",
           "in the restaurant",
           "in the store",
           "in the big supermarket",
           "A display counter"
           "A party or the celebration",
           "A gathering",
           "A happy hour, or a fun time",
           "A fun time",
           "There are some beer carton",
           "There are some Promotional Material: Signage, billboard, poster, standee ",
           "People is taking photo"]

text = tokenizer(context).to(device)



In [100]:
def define_context(image_path, model, device):
    raw_image = Image.open(image_path)
    image = preprocess(raw_image).unsqueeze(0).to(device)

    with torch.no_grad(), torch.cuda.amp.autocast():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
        max_value, max_index = torch.max(text_probs, dim=1)
        return max_value, max_index




In [102]:
image_path = '/content/dataset/FULL [Heineken Vietnam] Developer Resources/66550793_1706269652872.jpg'
max_value, max_index = define_context(image_path, model, device)
print(context[max_index], max_value)

in the store tensor([0.6091])


In [103]:
results_list = []

dataset_path = "/content/dataset/FULL [Heineken Vietnam] Developer Resources"

for filename in os.listdir(dataset_path):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust file extensions as needed
        image_path = os.path.join(dataset_path , filename)
        max_value, max_index = define_context(image_path, model, device)
        ct = context[max_index]
        object = {
            "filename": filename,
            "context": ct,
            "text_probs": max_value
        }
        results_list.append(object)



In [ ]:
import json


In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/HACKHCMC2024/context_result_list.json', 'w') as json_file:
    json.dump(results_list, json_file, indent=4)